In [14]:
import re
import spacy
import pandas as pd
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Descargar y cargar modelo de spaCy
tokenizer = spacy.load("es_core_news_sm")

# Cargar dataset desde archivo CSV
file_path = "data/reviews.csv"
df = pd.read_csv(file_path)

def preprocess_text(text):
    # 1. Convertir a minúsculas
    text = str(text).lower()
    
    # 2. Eliminar números y símbolos con expresión regular
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)  # Esto eliminará todo excepto letras y espacios
    
    # 3. Tokenización y lematización con spaCy
    doc = tokenizer(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in STOP_WORDS and not token.is_punct]
    
    # 4. Unir tokens procesados
    return ' '.join(tokens)

# Aplicar preprocesamiento a cada review
df['processed_review'] = df['review'].apply(preprocess_text)

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer() # param
X = vectorizer.fit_transform(df['processed_review'])
y = df['stars']

# Verificar si hay suficientes datos para dividir
if len(df) > 2:
    # División de datos para entrenamiento y prueba con estratificación
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    # Definir clasificadores
    classifiers = {
        "Logistic Regression": LogisticRegression(max_iter=1000),
        "Naive Bayes": MultinomialNB(),
        "SVM": SVC(),
        "Random Forest": RandomForestClassifier()
    }
    
    # Entrenar y evaluar cada clasificador
    for name, model in classifiers.items():
        model.fit(X_train_res, y_train_res)  # Entrenamos con los datos balanceados
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'{name} Accuracy: {accuracy:.2f}')
else:
    print("No hay suficientes datos para entrenar el modelo.")

Logistic Regression Accuracy: 0.63
Naive Bayes Accuracy: 0.47
SVM Accuracy: 0.55
Random Forest Accuracy: 0.56


In [ ]:
# Probar con stemming para comparar

In [21]:
import re
import numpy as np
import spacy
import pandas as pd
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

# Descargar y cargar modelo de spaCy
tokenizer = spacy.load("es_core_news_sm")

# Cargar dataset desde archivo CSV
file_path = "data/reviews.csv"
df = pd.read_csv(file_path)

def preprocess_text(text):
    # 1. Convertir a minúsculas
    text = str(text).lower()
    
    # 2. Eliminar números y símbolos con expresión regular
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)  # Esto eliminará todo excepto letras y espacios
    
    # 3. Tokenización y lematización con spaCy
    doc = tokenizer(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in STOP_WORDS and not token.is_punct]
    
    # 4. Unir tokens procesados
    return ' '.join(tokens)

# Aplicar preprocesamiento a cada review
df['processed_review'] = df['review'].apply(preprocess_text)

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_review'])
y = df['stars']

# Verificar si hay suficientes datos para dividir
if len(df) > 2:
    # División de datos para entrenamiento y prueba con estratificación
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    # Calcular los pesos de las clases para el Random Forest
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_res), y=y_train_res)
    class_weight_dict = dict(zip(np.unique(y_train_res), class_weights))
    
    # Crear el modelo Random Forest con los pesos de clase ajustados
    model_rf = RandomForestClassifier(random_state=42, class_weight=class_weight_dict)
    
    # Entrenar el modelo
    model_rf.fit(X_train_res, y_train_res)
    
    # Evaluación del modelo
    y_pred_rf = model_rf.predict(X_test)
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    print(f'Random Forest Accuracy con ajuste de pesos de clase: {accuracy_rf:.2f}')
else:
    print("No hay suficientes datos para entrenar el modelo.")


Random Forest Accuracy con ajuste de pesos de clase: 0.59


In [24]:
import re
import spacy
import pandas as pd
import numpy as np  # Importamos numpy
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier  # Importamos MLPClassifier

# Descargar y cargar modelo de spaCy
tokenizer = spacy.load("es_core_news_sm")

# Cargar dataset desde archivo CSV
file_path = "data/reviews.csv"
df = pd.read_csv(file_path)

def preprocess_text(text):
    # 1. Convertir a minúsculas
    text = str(text).lower()
    
    # 2. Eliminar números y símbolos con expresión regular
    text = re.sub(r'[^a-záéíóúüñ\s]', '', text)  # Esto eliminará todo excepto letras y espacios
    
    # 3. Tokenización y lematización con spaCy
    doc = tokenizer(text)
    tokens = [token.lemma_ for token in doc if token.lemma_ not in STOP_WORDS and not token.is_punct]
    
    # 4. Unir tokens procesados
    return ' '.join(tokens)

# Aplicar preprocesamiento a cada review
df['processed_review'] = df['review'].apply(preprocess_text)

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_review'])
y = df['stars']

# Verificar si hay suficientes datos para dividir
if len(df) > 2:
    # División de datos para entrenamiento y prueba con estratificación
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
    # Crear el modelo MLP sin el ajuste de pesos
    model_mlp = MLPClassifier(random_state=42, max_iter=500)
    
    # Entrenar el modelo
    model_mlp.fit(X_train_res, y_train_res)
    
    # Evaluación del modelo
    y_pred_mlp = model_mlp.predict(X_test)
    accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
    print(f'MLP Classifier Accuracy con SMOTE: {accuracy_mlp:.2f}')
else:
    print("No hay suficientes datos para entrenar el modelo.")


MLP Classifier Accuracy con SMOTE: 0.54
